In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
import pyspark.sql.functions as F



In [3]:
spark = SparkSession \
                .builder \
                .master("local[*]")\
                .appName("Exercicio Intro") \
                .getOrCreate()

23/05/10 00:41:52 WARN Utils: Your hostname, davyc-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp6s0)
23/05/10 00:41:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/10 00:41:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Questão 1

In [4]:
df_nomes = spark.read.csv("../nomes_aleatorios.txt", header=False, sep='\n')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



Questão 2

In [5]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nome')
df_nomes.show(10)

+-----------------+
|             Nome|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



Questões 3, 4 e 5

In [6]:
df_nomes = df_nomes.withColumn(
  "Escolaridade",
  F.array(
    F.lit("Fundamental"),
    F.lit("Médio"),
    F.lit("Superior"),
  ).getItem(
    (F.rand()*3).cast("int")
  )
).withColumn(
    "Pais",
    F.array(
    F.lit("Argentina"),
    F.lit("Bolivia"),
    F.lit("Brasil"),
    F.lit("Colombia"),
    F.lit("Chile"),
    F.lit("Venezuela"),
    F.lit("Guiana"),
    F.lit("Paraguai"),
    F.lit("Peru"),
    F.lit("Uruguai"),
    F.lit("Suriname"),
    F.lit("Equador"),
    F.lit("Guiana Francesa"),
  ).getItem(
    (F.rand()*13).cast("int")
  )
).withColumn(
    'AnoNascimento',
    F.round(F.rand()*(2010-1945)+1945)

)

df_nomes.show(100)


/opt/spark/python/pyspark/sql/column.py:458: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


+--------------------+------------+---------------+-------------+
|                Nome|Escolaridade|           Pais|AnoNascimento|
+--------------------+------------+---------------+-------------+
|      Frances Bennet|    Superior|       Paraguai|       1968.0|
|       Jamie Russell| Fundamental|       Colombia|       2000.0|
|      Edward Kistler| Fundamental|        Equador|       1988.0|
|       Sheila Maurer| Fundamental|      Argentina|       1977.0|
|    Donald Golightly|    Superior|       Paraguai|       1962.0|
|          David Gray| Fundamental|       Colombia|       1995.0|
|         Joy Bennett| Fundamental|         Brasil|       1976.0|
|         Paul Kriese| Fundamental|         Guiana|       2006.0|
|   Berniece Ornellas|       Médio|        Equador|       1984.0|
|       Brian Farrell| Fundamental|        Equador|       1970.0|
|      Kara Mcelwaine|       Médio|      Venezuela|       1991.0|
|       Tracy Herring|       Médio|       Paraguai|       1977.0|
|     Howa

Questão 6

In [7]:
df_select = df_nomes.select("Nome", "AnoNascimento").filter(df_nomes['AnoNascimento'] > 2000)
df_select.show(10)

+----------------+-------------+
|            Nome|AnoNascimento|
+----------------+-------------+
|     Paul Kriese|       2006.0|
|    David Medina|       2010.0|
|  Amanda Gravitt|       2003.0|
| Kenneth Rayburn|       2002.0|
|  Mary Dillahunt|       2003.0|
|     Donald Vogt|       2006.0|
|   Lynne Dustman|       2009.0|
|   George Miller|       2003.0|
|Maurice Blizzard|       2004.0|
|    Curtis Sutch|       2007.0|
+----------------+-------------+
only showing top 10 rows



Questão 7

In [8]:
df_nomes.createOrReplaceTempView("pessoas")
spark.sql("select Nome, AnoNascimento from pessoas where AnoNascimento > 2000").show(10)

+----------------+-------------+
|            Nome|AnoNascimento|
+----------------+-------------+
|     Paul Kriese|       2006.0|
|    David Medina|       2010.0|
|  Amanda Gravitt|       2003.0|
| Kenneth Rayburn|       2002.0|
|  Mary Dillahunt|       2003.0|
|     Donald Vogt|       2006.0|
|   Lynne Dustman|       2009.0|
|   George Miller|       2003.0|
|Maurice Blizzard|       2004.0|
|    Curtis Sutch|       2007.0|
+----------------+-------------+
only showing top 10 rows



Questão 8

In [9]:
df_nomes.filter((df_nomes['AnoNascimento'] >= 1980) & (df_nomes['AnoNascimento'] <= 1994)).count()


2305770

Questão 9

In [33]:
spark.sql("select count(Nome) as Millennials from pessoas where anoNascimento >= 1980 and anoNascimento <=1994").show()

+-----------+
|Millennials|
+-----------+
|    2305770|
+-----------+



Questão 10


In [32]:
geracoes = spark.sql(
    """
    SELECT
        sum(case when anoNascimento >= 1944 and anoNascimento <=1964 then 1 else 0 end ) as Geracao,        
        Pais

    from pessoas
    group by (Pais)
    order by Pais, Geracao desc
    """
)

geracoes.show()

+-------+---------------+
|Geracao|           Pais|
+-------+---------------+
| 231053|      Argentina|
| 231384|        Bolivia|
| 230888|         Brasil|
| 231130|          Chile|
| 231140|       Colombia|
| 231862|        Equador|
| 231056|         Guiana|
| 230287|Guiana Francesa|
| 230457|       Paraguai|
| 230746|           Peru|
| 231355|       Suriname|
| 231215|        Uruguai|
| 230744|      Venezuela|
+-------+---------------+

